<a href="https://colab.research.google.com/github/AdrianaUP/News-query_RPP-lab/blob/main/notebooks/task1_rpp_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Notebook inicializado para Task 1")


Notebook inicializado para Task 1


In [2]:
pip install feedparser


In [3]:
import feedparser
import pandas as pd

rss_url = "https://rpp.pe/rss"
feed = feedparser.parse(rss_url)

news_items = []
for entry in feed.entries[:50]:
    news_items.append({
        "title": entry.title,
        "description": entry.description,
        "link": entry.link,
        "published": entry.published
    })

df_news = pd.DataFrame(news_items)
df_news.head()


,title,description,link,published
0,Perú perdió 4-1 ante Colombia en su debut en l...,La Selección Peruana cayó estrepitosamente en ...,https://rpp.pe/futbol/seleccion-peruana/peru-v...,"Fri, 24 Oct 2025 20:30:14 -0500"
1,Comisión de Energía y Minas acordó conformar m...,"Con 14 votos y 8 en contra, dicho grupo de tra...",https://rpp.pe/politica/congreso/comision-de-e...,"Fri, 24 Oct 2025 20:23:38 -0500"
2,Trujillo: Policía resguarda rutas de empresas ...,"En diálogo con RPP, el coronel Jorge Porras, j...",https://rpp.pe/peru/actualidad/trujillo-polici...,"Fri, 24 Oct 2025 20:13:58 -0500"
3,Temblor en Chile hoy 24 de octubre: Epicentro ...,¿Cuál fue el último Temblor en Chile hoy 24 de...,https://rpp.pe/mundo/chile/temblor-en-chile-ho...,"Fri, 24 Oct 2025 05:09:45 -0500"
4,Lambayeque: ciudadano belga rescata y libera a...,Philippe André es conocido en Lambayeque por s...,https://rpp.pe/peru/lambayeque/lambayeque-ciud...,"Fri, 24 Oct 2025 19:53:30 -0500"


In [4]:
pip install tiktoken


In [5]:
import tiktoken

# Usaremos el codificador de OpenAI compatible con modelos como GPT-3.5
encoding = tiktoken.get_encoding("cl100k_base")

# Tomamos el texto completo de la primera noticia (título + descripción)
sample_text = df_news.loc[0, "title"] + " " + df_news.loc[0, "description"]

# Tokenizamos
tokens = encoding.encode(sample_text)

# Mostramos cantidad de tokens
print(f"Número de tokens: {len(tokens)}")


Número de tokens: 49


In [6]:
def count_tokens(text):
    return len(encoding.encode(text))

df_news["full_text"] = df_news["title"] + " " + df_news["description"]
df_news["num_tokens"] = df_news["full_text"].apply(count_tokens)

df_news[["title", "num_tokens"]].head()


,title,num_tokens
0,Perú perdió 4-1 ante Colombia en su debut en l...,49
1,Comisión de Energía y Minas acordó conformar m...,96
2,Trujillo: Policía resguarda rutas de empresas ...,72
3,Temblor en Chile hoy 24 de octubre: Epicentro ...,74
4,Lambayeque: ciudadano belga rescata y libera a...,55


In [7]:
pip install -U numpy torch transformers==4.33.2 huggingface_hub==0.17.1 sentence-transformers==2.2.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.9/314.9 kB 14.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 61.0 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: ERROR: Failed to build installable wheels for some py